<a href="https://colab.research.google.com/github/Noman-ashraf1/Project/blob/main/LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# Libraries



In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as numpy
import matplotlib.pyplot as plt
import seaborn as sns

from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import tensorflow
import keras


# Load the Dataset


In [6]:
df = pd.read_csv("/content/Combined Data.csv")

new_df = df.sample(n=10000, random_state=42).reset_index(drop=True)
new_df.drop("Unnamed: 0", axis=1, inplace=True)
new_df.head()


,statement,status
0,Just as the the title says. I feel like one is...,Depression
1,a blackened sky encroached tugging behind it m...,Depression
2,"It gives you insomnia, which in turn makes you...",Depression
3,"Hello all, I'm a new submitter to this channel...",Normal
4,Thank God the CB is over for Eid,Normal


# Preprocessing

 Data

In [7]:
import nltk
from nltk.corpus import stopwords
import re

# Download stopwords (only needed the first time)
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
def clean_text(text):
  text = str(text).lower()
  text = re.sub(r'[^a-zA-Z\s]', '', text)
  words = text.split()
  words = [word for word in words if word  not in stop_words]
  return " ".join(words)

new_df["statement"] = new_df["statement"].apply(clean_text)
new_df


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,statement,status
0,title says feel like one hyper alot attention ...,Depression
1,blackened sky encroached tugging behind depres...,Depression
2,gives insomnia turn makes depression worse day...,Depression
3,hello im new submitter channel went nights ago...,Normal
4,thank god cb eid,Normal
...,...,...
9995,right feel passively suicidal feel like trying...,Depression
9996,call assistance pol hoping investigated turn w...,Normal
9997,came back college assignment really pile like ...,Normal
9998,por q siempre matan jyp culiadas jakdjs,Normal


# Imbalance dataset


In [8]:
new_df["status"].value_counts()

,count
status,
Normal,3124
Depression,2931
Suicidal,1890
Anxiety,739
Bipolar,552
Stress,525
Personality disorder,239


In [9]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(sampling_strategy='auto')
x = new_df.drop("status", axis=1)
y = new_df['status']
sample_x, sample_y = ros.fit_resample(x,y)
data = pd.concat([sample_x, sample_y], axis=1)
data

,statement,status
0,title says feel like one hyper alot attention ...,Depression
1,blackened sky encroached tugging behind depres...,Depression
2,gives insomnia turn makes depression worse day...,Depression
3,hello im new submitter channel went nights ago...,Normal
4,thank god cb eid,Normal
...,...,...
21863,wish upon noone depression sick curse,Suicidal
21864,need help mentally unwell help,Suicidal
21865,things gotten bad body trying make laugh inste...,Suicidal
21866,thinking since childhood recent past traumas e...,Suicidal


In [10]:
data['status'].value_counts()

,count
status,
Depression,3124
Normal,3124
Suicidal,3124
Anxiety,3124
Bipolar,3124
Stress,3124
Personality disorder,3124


# Label encoding

In [11]:
lab = LabelEncoder()
data['label'] = lab.fit_transform(data['status'])
data



,statement,status,label
0,title says feel like one hyper alot attention ...,Depression,2
1,blackened sky encroached tugging behind depres...,Depression,2
2,gives insomnia turn makes depression worse day...,Depression,2
3,hello im new submitter channel went nights ago...,Normal,3
4,thank god cb eid,Normal,3


# Tokenization


In [12]:
#maximum length in a sigle line
maxl = max([len(x) for x in data['statement']])

In [13]:
#split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(data['statement'], data['label'], test_size=0.2)

In [14]:
token = BertTokenizer.from_pretrained('bert-base-uncased')
train_encode = token(list(X_train), padding=True, truncation=True, max_length=200, return_tensors='tf')
test_encode = token(list(X_test), padding=True, truncation=True, max_length=200, return_tensors='tf')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [15]:
from datasets import Dataset
train_data = Dataset.from_dict(
    {
        'input_ids' : train_encode['input_ids'],
        'attention_mask': train_encode['attention_mask'],
        'label': y_train.tolist()
    }
)
test_data = Dataset.from_dict(
    {
        'input_ids' : test_encode['input_ids'],
        'attention_mask': test_encode['attention_mask'],
        'label': y_test.tolist()
    }
)


# Fine tune

In [16]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=7)
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=16,  # batch size for training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,                # how often to log training info
    evaluation_strategy="epoch",     # evaluate after each epoch
    save_strategy="epoch",          # save model after each epoch
    load_best_model_at_end=True,     # load best model when finished training
    metric_for_best_model='eval_loss',  # evaluation metric for best model
    greater_is_better=True,           # whether the metric should be maximized
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_data,
    eval_dataset = test_data


)
trainer.train()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.345800,0.502338
2,0.264500,0.270288
3,0.144600,0.310033


TrainOutput(global_step=3282, training_loss=0.41334398768247443, metrics={'train_runtime': 2072.7345, 'train_samples_per_second': 25.32, 'train_steps_per_second': 1.583, 'total_flos': 5394224342340000.0, 'train_loss': 0.41334398768247443, 'epoch': 3.0})

In [18]:
import numpy as np
predictions, labels, extra_info = trainer.predict(test_data)
predicted_labels = np.argmax(predictions, axis=1)


In [19]:
print("Number of classes in y_test:", len(set(y_test)))
print("Number of classes in predicted_labels:", len(set(predicted_labels)))


Number of classes in y_test: 7
Number of classes in predicted_labels: 7


In [20]:
data['label'].value_counts()

,count
label,
2,3124
3,3124
6,3124
0,3124
1,3124
5,3124
4,3124


In [21]:
target_names = ['class_0', 'class_1', 'class_2', 'class_3', 'class_4', 'class_5', 'class_6']


In [22]:
from sklearn.metrics import classification_report

# Assuming y_test and predicted_labels are already defined
print(classification_report(y_test, predicted_labels, target_names=target_names))



              precision    recall  f1-score   support

     class_0       0.93      0.96      0.95       630
     class_1       0.96      0.91      0.93       627
     class_2       0.53      0.77      0.63       575
     class_3       0.97      0.80      0.88       678
     class_4       0.88      1.00      0.93       631
     class_5       0.83      0.93      0.88       609
     class_6       0.76      0.42      0.54       624

    accuracy                           0.83      4374
   macro avg       0.84      0.83      0.82      4374
weighted avg       0.84      0.83      0.82      4374



#prediction

In [92]:
def detect(text):
    text = clean_text(text)
    inputs = token(text, padding=True, truncation=True, max_length=200, return_tensors='tf')
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_label_index = tf.argmax(logits, axis=1).numpy().item()
    predicted_label = data['label'].inverse_transform([predicted_label_index])[0]
    return predicted_label